This notebook was prepared by:

Jack Brennen (jpb9088@nyu.edu)

Fabrizio Cabrera (fc2250@nyu.edu)

Alex Ferrante (af3913@nyu.edu)

Graduate School of Arts and Sciences (GSAS) at New York University (NYU)

May 2022

# Predicting Daily Stock Outcomes - An Application of Various Machine Learning Classifiers  (can change to something sexier)

The ability to predict the movement of a stock was once far fetched, however, thanks to advancements in machine learning techniques, there is reason to
believe short-run predictions are possible, within reason. So what makes stock movement prediction tricky? Well, several factors, but one which is most obscure is market noise. With most persons' having access to the internet, news spreads quickly. This means consumers' sentiments are subject to changing quickly which results in short-run market fluctuations. Exogenous shocks to the market come in many forms and all of which are reported on. With the rise of internet news platforms, these sources server as venues for market particpants to consume stock information which subsequtnely informs short-run behavior. 

Since we believe that capturing trends in agents news consumption is valuable in explaining short-run movements in stocks, this project will include data from Google Trends and Wikipedia API's to serve as indexes to inform the frequency and popularity of stock news. In addition to data collected from Google Trends and Wikipedia API's we also collect general stock data (open, close, volumem ect.). Through the use of random forrests, artificial neural networks (ANN), and recursive neural networks (RNN) we will contruct a binary-classification task and see how accurately we can model short-run stock movement. While there are an array of targets we could test, the target for the procedding models will be whether the stock closing price will increase tomorrow (1) or whether it will decrease (0). 

Given constraints in time and computing power, we will collect data from XXX to XXX for Ford, XXX, and XXX. 


## Pulling Together the Data

As the saying goes, garbage in, garbage out. Thus, this sections reviews the necessary steps taken to aquire the news data from Google Trends and Wikipedia. Given the amount of time and computing power used in pulling the data, the following functions used will be presented in a markdown cell to share the code used.

Big scraper and other data functions here

After pulling all of the necessary data, there are a few things worth noting that took place. (will mention the creation of the features variables and other data transformation that took place)